# Step 2: GPT Model - Transformer 架构

## 学习目标

1. 理解 Embedding（词嵌入 + 位置嵌入）
2. **动手实现** Multi-Head Self-Attention
3. **动手实现** Feed-Forward Network (MLP)
4. **动手实现** 完整的 GPT 模型

## 学习方式

1. 在这个 Notebook 中学习概念、运行代码、查看可视化
2. 去 `model_exercise.py` 完成 TODO 部分的代码
3. 回到这里验证你的实现
4. 卡住了？查看 `model_solution.py` 中的答案

---

## 1. GPT 架构概览

```
输入 Token IDs: [1, 45, 23, 89, 12]
        ↓
┌─────────────────────────────────────┐
│  Token Embedding + Position Embedding │
└─────────────────────────────────────┘
        ↓
┌─────────────────────────────────────┐
│         Transformer Block × N         │
│  ┌─────────────────────────────────┐  │
│  │  LayerNorm → Attention → +残差   │  │
│  │  LayerNorm → MLP → +残差         │  │
│  └─────────────────────────────────┘  │
└─────────────────────────────────────┘
        ↓
┌─────────────────────────────────────┐
│    Final LayerNorm → LM Head          │
└─────────────────────────────────────┘
        ↓
输出 Logits: [vocab_size] 的概率分布
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# 设置随机种子
torch.manual_seed(42)

---

## 2. Embedding 层

将离散的 token ID 转换为连续的向量表示。

In [ ]:
# 参数设置
vocab_size = 1000   # 词表大小
n_embd = 64         # 嵌入维度
block_size = 32     # 最大序列长度

# Token Embedding: 每个 token ID 映射为一个向量
tok_emb = nn.Embedding(vocab_size, n_embd)

# Position Embedding: 每个位置映射为一个向量
pos_emb = nn.Embedding(block_size, n_embd)

# 模拟输入
tokens = torch.tensor([[10, 20, 30, 40]])  # [1, 4]
positions = torch.arange(4)  # [0, 1, 2, 3]

# 查看 embedding 结果
print(f"Token IDs: {tokens}")
print(f"Token Embedding shape: {tok_emb(tokens).shape}")
print(f"Position Embedding shape: {pos_emb(positions).shape}")

# 最终输入 = Token Embedding + Position Embedding
x = tok_emb(tokens) + pos_emb(positions)
print(f"\n最终输入 shape: {x.shape}")

---

## 3. Self-Attention 机制

Self-Attention 的核心：让每个 token "看到"序列中的其他 token

### 3.1 Q, K, V 的含义

- **Query (Q)**: "我在找什么？"
- **Key (K)**: "我有什么？" 
- **Value (V)**: "我的内容是什么？"

计算过程：
```
Attention = softmax(Q @ K^T / sqrt(d_k)) @ V
```

In [ ]:
# 手动实现简单的 Self-Attention（单头）
B, T, C = 1, 4, 64  # batch=1, seq_len=4, dim=64
x = torch.randn(B, T, C)

# Q, K, V 投影
W_q = nn.Linear(C, C, bias=False)
W_k = nn.Linear(C, C, bias=False)
W_v = nn.Linear(C, C, bias=False)

Q = W_q(x)  # [1, 4, 64]
K = W_k(x)  # [1, 4, 64]
V = W_v(x)  # [1, 4, 64]

print(f"Q shape: {Q.shape}")
print(f"K shape: {K.shape}")
print(f"V shape: {V.shape}")

In [ ]:
# 计算注意力分数
# att = Q @ K^T / sqrt(d_k)
att = Q @ K.transpose(-2, -1) / math.sqrt(C)
print(f"注意力分数 shape: {att.shape}")  # [1, 4, 4]
print(f"\n注意力分数矩阵:")
print(att[0].detach().numpy().round(2))

### 3.2 因果掩码（Causal Mask）

GPT 是自回归模型：**每个 token 只能看到它之前的 token**

通过上三角掩码实现：
```
     pos0  pos1  pos2  pos3
pos0  ✓     ✗     ✗     ✗    (只能看自己)
pos1  ✓     ✓     ✗     ✗    (看 pos0, pos1)
pos2  ✓     ✓     ✓     ✗    (看 pos0, pos1, pos2)
pos3  ✓     ✓     ✓     ✓    (看所有)
```

In [ ]:
# 创建因果掩码
mask = torch.triu(torch.ones(T, T), diagonal=1).bool()
print("因果掩码（True 表示要屏蔽的位置）:")
print(mask.int().numpy())

# 应用掩码：将上三角设为 -inf
att_masked = att.masked_fill(mask, float('-inf'))
print(f"\n应用掩码后:")
print(att_masked[0].detach().numpy().round(2))

In [ ]:
# Softmax 后，-inf 变成 0
att_weights = F.softmax(att_masked, dim=-1)
print("Softmax 后的注意力权重:")
print(att_weights[0].detach().numpy().round(2))

# 最终输出
output = att_weights @ V
print(f"\n输出 shape: {output.shape}")

### 3.3 练习：实现 CausalSelfAttention

现在去 `model_exercise.py`，完成 **TODO 3a, 3b, 3c**：

- 3a: 计算注意力分数 `att = Q @ K^T / sqrt(d_k)`
- 3b: 应用因果掩码
- 3c: Softmax + Dropout + 与 V 相乘

In [ ]:
# 重新导入测试
import importlib
import model_exercise
importlib.reload(model_exercise)
from model_exercise import CausalSelfAttention, GPTConfig

config = GPTConfig(n_embd=128, n_head=4)
attn = CausalSelfAttention(config)

x = torch.randn(2, 32, 128)
try:
    out = attn(x)
    print(f"输入 shape: {x.shape}")
    print(f"输出 shape: {out.shape}")
    print("\n✅ CausalSelfAttention 实现正确!")
except NotImplementedError as e:
    print(f"⚠️ {e}")

---

## 4. MLP（前馈网络）

结构：Linear → GELU → Linear

先扩展到 4 倍维度，再压缩回来。这是 Transformer 中的"思考"部分。

In [ ]:
# MLP 结构演示
n_embd = 128

c_fc = nn.Linear(n_embd, 4 * n_embd)    # 扩展到 512
gelu = nn.GELU()
c_proj = nn.Linear(4 * n_embd, n_embd)  # 压缩回 128

x = torch.randn(2, 32, 128)
print(f"输入: {x.shape}")
print(f"扩展: {c_fc(x).shape}")
print(f"GELU: {gelu(c_fc(x)).shape}")
print(f"输出: {c_proj(gelu(c_fc(x))).shape}")

### 4.1 练习：实现 MLP

去 `model_exercise.py`，完成 **TODO 1**：实现 MLP 的 forward 方法

In [ ]:
# 测试 MLP
importlib.reload(model_exercise)
from model_exercise import MLP, GPTConfig

config = GPTConfig(n_embd=128)
mlp = MLP(config)

x = torch.randn(2, 32, 128)
try:
    out = mlp(x)
    print(f"输入 shape: {x.shape}")
    print(f"输出 shape: {out.shape}")
    print("\n✅ MLP 实现正确!")
except NotImplementedError as e:
    print(f"⚠️ {e}")

---

## 5. Transformer Block

结构（Pre-LayerNorm + Residual）:
```python
x = x + Attention(LayerNorm(x))  # 残差连接
x = x + MLP(LayerNorm(x))        # 残差连接
```

残差连接让梯度可以直接流过，使深层网络更容易训练。

### 5.1 练习：实现 TransformerBlock

去 `model_exercise.py`，完成 **TODO 2**：实现 TransformerBlock 的 forward 方法

In [ ]:
# 测试 TransformerBlock
importlib.reload(model_exercise)
from model_exercise import TransformerBlock, GPTConfig

config = GPTConfig(n_embd=128, n_head=4)
block = TransformerBlock(config)

x = torch.randn(2, 32, 128)
try:
    out = block(x)
    print(f"输入 shape: {x.shape}")
    print(f"输出 shape: {out.shape}")
    print("\n✅ TransformerBlock 实现正确!")
except NotImplementedError as e:
    print(f"⚠️ {e}")

---

## 6. 完整的 GPT 模型

### 6.1 练习：实现 GPT forward

去 `model_exercise.py`，完成 **TODO 4a, 4b, 4c**：

- 4a: Token Embedding + Position Embedding
- 4b: 通过所有 Transformer Blocks
- 4c: Final LayerNorm + LM Head

In [ ]:
# 测试完整 GPT
importlib.reload(model_exercise)
from model_exercise import GPT, GPTConfig

config = GPTConfig(
    vocab_size=1000,
    block_size=64,
    n_embd=128,
    n_head=4,
    n_layer=4
)

try:
    model = GPT(config)
    print(f"参数量: {model.get_num_params()/1e6:.2f}M")
    
    # 测试前向传播
    idx = torch.randint(0, 1000, (2, 32))
    logits, loss = model(idx, idx)
    
    print(f"\n输入 shape: {idx.shape}")
    print(f"输出 logits shape: {logits.shape}")
    print(f"Loss: {loss.item():.4f}")
    print("\n✅ GPT 实现正确!")
except NotImplementedError as e:
    print(f"⚠️ {e}")

### 6.2 测试生成

In [ ]:
# 测试生成（如果 GPT 实现正确）
try:
    start = torch.tensor([[1, 2, 3]])  # 起始 tokens
    generated = model.generate(start, max_new_tokens=10, temperature=1.0)
    print(f"起始: {start.tolist()}")
    print(f"生成: {generated.tolist()}")
except:
    print("请先完成 GPT 的实现")

---

## 7. 参数量分析

GPT 参数量主要来自：
- Embedding: vocab_size × n_embd
- Attention: 4 × n_embd² per layer (Q, K, V, Output)
- MLP: 8 × n_embd² per layer (扩展 + 压缩)

**估算公式**: 约 12 × n_layer × n_embd² 个参数

In [ ]:
# 不同规模的 GPT
configs = [
    ("Small", GPTConfig(vocab_size=6400, n_embd=512, n_head=8, n_layer=8)),
    ("Medium", GPTConfig(vocab_size=6400, n_embd=768, n_head=12, n_layer=12)),
    ("Large", GPTConfig(vocab_size=6400, n_embd=1024, n_head=16, n_layer=24)),
]

print("不同规模的 GPT 参数量:")
print("-" * 50)
for name, config in configs:
    # 估算公式
    estimated = 12 * config.n_layer * config.n_embd ** 2
    print(f"{name}: n_embd={config.n_embd}, n_layer={config.n_layer}")
    print(f"  估算: ~{estimated/1e6:.1f}M 参数")

---

## 8. 运行完整测试

In [ ]:
!python model_exercise.py

---

## 9. 验证清单

完成本步骤后，你应该能够：

- [ ] 画出 GPT 的架构图
- [ ] 解释 Self-Attention 的计算过程（Q, K, V）
- [ ] 解释为什么需要因果掩码
- [ ] 实现 MLP、TransformerBlock、GPT
- [ ] 计算给定配置的模型参数量

---

## 下一步

理解模型架构后，进入 [Step 3: Pretrain](../step3_pretrain/) 学习如何预训练这个模型。